In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import copy


In [2]:
#DEVICE SETUP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  IMAGE LOADING & PREPROCESSING

def load_image(img_path, max_size=400, shape=None):
    image = Image.open(img_path).convert('RGB')
    size = max_size if max(image.size) > max_size else max(image.size)
    if shape:
        size = shape
    in_transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))])
    image = in_transform(image).unsqueeze(0)
    return image.to(device)

# LOAD IMAGES
# Replace 'content.jpg' and 'style.jpg' with your own file paths
content = load_image("content.jpg")
style = load_image("style.jpg", shape=content.shape[-2:])


# 📊 DISPLAY IMAGES

def im_convert(tensor):
    image = tensor.to("cpu").clone().detach().squeeze(0)
    image = image.numpy().transpose(1, 2, 0)
    image = image * (0.229, 0.224, 0.225) + (0.485, 0.456, 0.406)
    return image.clip(0, 1)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Content Image")
plt.imshow(im_convert(content))
plt.subplot(1, 2, 2)
plt.title("Style Image")
plt.imshow(im_convert(style))
plt.show()
# 📦 LOAD PRETRAINED VGG19 MODEL

vgg = models.vgg19(pretrained=True).features
for param in vgg.parameters():
    param.requires_grad_(False)
vgg.to(device)

# -----------------------
# 🔍 FEATURE EXTRACTION
# -----------------------
style_layers = ['0', '5', '10', '19', '28']
content_layers = ['21']

def get_features(image, model, layers=None):
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if layers and name in layers:
            features[name] = x
    return features

def gram_matrix(tensor):
    b, c, h, w = tensor.size()
    tensor = tensor.view(c, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram


# EXTRACT FEATURES

content_features = get_features(content, vgg, content_layers)
style_features = get_features(style, vgg, style_layers)
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# create target image

target = content.clone().requires_grad_(True).to(device)


#  NEURAL STYLE TRANSFER
style_weight = 1e6
content_weight = 1e0
optimizer = optim.Adam([target], lr=0.003)

steps = 2000
for i in range(1, steps + 1):
    target_features = get_features(target, vgg)
    content_loss = torch.mean((target_features['21'] - content_features['21']) ** 2)

    style_loss = 0
    for layer in style_layers:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        style_gram = style_grams[layer]
        layer_loss = torch.mean((target_gram - style_gram) ** 2)
        style_loss += layer_loss / (target_feature.shape[1] ** 2)

    total_loss = content_weight * content_loss + style_weight * style_loss

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if i % 500 == 0:
        print(f"Step {i}, Total loss: {total_loss.item():.4f}")
#SHOW RESULT

plt.figure(figsize=(10, 5))
plt.title("Styled Image")
plt.imshow(im_convert(target))
plt.axis('off')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'content.jpg'